In [1]:
# install dataset helper
!pip install -q ucimlrepo

# essentials
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier as SklearnDT
import joblib
import random
import math
import time
import pprint
np.random.seed(42)
random.seed(42)


In [2]:
adult = fetch_ucirepo(id=2)    # Adult dataset from UCI via ucimlrepo
X = adult.data.features.copy()    # pandas DataFrame
y = adult.data.targets.copy()     # pandas DataFrame or Series
# target column name might vary; ensure it's a 1D array/Series
if isinstance(y, pd.DataFrame):
    y = y.iloc[:,0]
print("Features shape:", X.shape)
print("Target shape:", y.shape)
X.head()


Features shape: (48842, 14)
Target shape: (48842,)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [3]:
# 1) Inspect missing entries (dataset may use '?' or NaN)
display(X.head())
print("Unique values (example) for workclass:", X['workclass'].unique())

# Replace ' ?' or '?' with np.nan then drop or impute
X = X.replace(' ?', np.nan).replace('?', np.nan)

# Drop rows with missing values (simple approach). You may impute instead.
df = pd.concat([X, y.rename('target')], axis=1)
print("Original rows:", df.shape[0])
df = df.dropna().reset_index(drop=True)
print("After dropping missing rows:", df.shape[0])

# Separate back
X = df.drop(columns=['target'])
y = df['target']

# Label-encode categorical variables
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
print("Categorical columns:", cat_cols)
print("Numeric columns:", num_cols)

encoders = {}
for c in cat_cols:
    le = LabelEncoder()
    X[c] = le.fit_transform(X[c].astype(str))
    encoders[c] = le

# Encode target if needed (e.g., '<=50K' vs '>50K')
if y.dtype == 'object' or y.dtype.name == 'category':
    le_y = LabelEncoder()
    y = le_y.fit_transform(y.astype(str))
    # store mapping:
    target_mapping = {i:lab for i,lab in enumerate(le_y.classes_)}
else:
    target_mapping = None

print("Class distribution:", np.bincount(y))


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


Unique values (example) for workclass: ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked' nan]
Original rows: 48842
After dropping missing rows: 45222
Categorical columns: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
Numeric columns: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
Class distribution: [22654 11360  7508  3700]


In [4]:
# 60% train, 20% val, 20% test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# split trainval into train (75% of trainval -> 60% total) and val (25% -> 20% total)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=42, stratify=y_trainval)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (27132, 14) Val: (9045, 14) Test: (9045, 14)


In [5]:
# Decision Tree from scratch (binary splits)
class TreeNode:
    def __init__(self, is_leaf=False, prediction=None, feature=None, threshold=None, left=None, right=None, depth=0, samples_indices=None):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.depth = depth
        self.samples_indices = samples_indices  # optional, for pruning traversal

def entropy(y):
    # y is 1D array-like of labels
    vals, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    # avoid log(0)
    return -np.sum([p * np.log2(p) for p in probs if p > 0.0])

def gini(y):
    vals, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return 1.0 - np.sum(probs**2)

def impurity(y, measure='gini'):
    return gini(y) if measure == 'gini' else entropy(y)

def weighted_impurity(y_left, y_right, measure='gini'):
    n = len(y_left) + len(y_right)
    return (len(y_left) * impurity(y_left, measure) + len(y_right) * impurity(y_right, measure)) / n

def best_split(X, y, feature_indices=None, measure='gini', min_samples_split=2):
    # returns feature, threshold, impurity_after_split, left_idx, right_idx
    n_samples, n_features = X.shape
    if feature_indices is None:
        feature_indices = range(n_features)

    best = {'gain': -np.inf, 'feature': None, 'threshold': None, 'left_idx': None, 'right_idx': None, 'impurity': None}
    current_imp = impurity(y, measure)

    for fi in feature_indices:
        col = X[:, fi]
        # if numeric (continuous encoded values) - try midpoints
        unique_vals = np.unique(col)
        if len(unique_vals) <= 1:
            continue
        # candidate thresholds: midpoints for sorted unique numeric values
        thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2.0
        # For label-encoded categorical where integer categories exist,
        # thresholds above also handle splits like <= category_k vs > category_k
        for thr in thresholds:
            left_idx = np.where(col <= thr)[0]
            right_idx = np.where(col > thr)[0]
            if len(left_idx) < min_samples_split or len(right_idx) < min_samples_split:
                continue
            imp_after = weighted_impurity(y[left_idx], y[right_idx], measure)
            gain = current_imp - imp_after
            if gain > best['gain']:
                best.update({'gain': gain, 'feature': fi, 'threshold': thr, 'left_idx': left_idx, 'right_idx': right_idx, 'impurity': imp_after})
    return best

def majority_class(y):
    vals, counts = np.unique(y, return_counts=True)
    return vals[np.argmax(counts)]

def build_tree(X, y, measure='gini', max_depth=None, min_samples_split=2, min_impurity_decrease=0.0, depth=0):
    # X: numpy array, y: numpy array
    n_samples = X.shape[0]
    node_prediction = majority_class(y)
    # stopping criteria
    if len(np.unique(y)) == 1:
        return TreeNode(is_leaf=True, prediction=node_prediction, depth=depth)
    if max_depth is not None and depth >= max_depth:
        return TreeNode(is_leaf=True, prediction=node_prediction, depth=depth)
    if n_samples < min_samples_split:
        return TreeNode(is_leaf=True, prediction=node_prediction, depth=depth)

    best = best_split(X, y, measure=measure, min_samples_split=min_samples_split)
    if best['feature'] is None or best['gain'] <= min_impurity_decrease:
        return TreeNode(is_leaf=True, prediction=node_prediction, depth=depth)

    # build children
    left = build_tree(X[best['left_idx']], y[best['left_idx']], measure=measure,
                      max_depth=max_depth, min_samples_split=min_samples_split,
                      min_impurity_decrease=min_impurity_decrease, depth=depth+1)
    right = build_tree(X[best['right_idx']], y[best['right_idx']], measure=measure,
                      max_depth=max_depth, min_samples_split=min_samples_split,
                      min_impurity_decrease=min_impurity_decrease, depth=depth+1)

    node = TreeNode(is_leaf=False, prediction=node_prediction, feature=best['feature'], threshold=best['threshold'], left=left, right=right, depth=depth)
    return node

def predict_single(node, x):
    while not node.is_leaf:
        if x[node.feature] <= node.threshold:
            node = node.left
        else:
            node = node.right
    return node.prediction

def predict_tree(node, X):
    return np.array([predict_single(node, x) for x in X])

# Utility: traverse internal nodes and collect them (for post-pruning)
def collect_internal_nodes(node, parent=None, attr_name=None, nodes=None):
    if nodes is None:
        nodes = []
    if node is None:
        return nodes
    if not node.is_leaf:
        nodes.append((node, parent, attr_name))
        collect_internal_nodes(node.left, node, 'left', nodes)
        collect_internal_nodes(node.right, node, 'right', nodes)
    return nodes

# Post-pruning: reduced-error pruning using validation set
def reduced_error_prune(root, X_val, y_val):
    """
    Iteratively try replacing each internal node with a leaf (majority class)
    and keep the change if validation accuracy does not decrease.
    Repeat until no improvement possible.
    """
    improved = True
    best_acc = metrics.accuracy_score(y_val, predict_tree(root, X_val))
    print(f"[Prune] Starting validation acc: {best_acc:.4f}")
    while improved:
        improved = False
        internal_nodes = collect_internal_nodes(root)
        # try nodes in bottom-up order (leaf-most first). We already get nodes in pre-order,
        # so sort by depth descending to get bottom-up.
        internal_nodes_sorted = sorted(internal_nodes, key=lambda t: t[0].depth, reverse=True)
        for node, parent, attr_name in internal_nodes_sorted:
            # backup subtree
            backup = (node.is_leaf, node.prediction, node.feature, node.threshold, node.left, node.right)
            # replace node with leaf
            maj = node.prediction  # note: node.prediction equals majority of that node's training subset if recorded; if not, compute
            # to be safe compute majority from scratch is not trivial since we didn't store indices; but node.prediction was set during training as majority class.
            new_leaf = TreeNode(is_leaf=True, prediction=maj, depth=node.depth)
            if parent is None:
                # pruning root: replace entire tree
                root = new_leaf
            else:
                setattr(parent, attr_name, new_leaf)
            # evaluate
            ypred = predict_tree(root, X_val)
            acc = metrics.accuracy_score(y_val, ypred)
            if acc >= best_acc - 1e-12:
                # accept pruning
                improved = True
                best_acc = acc
                print(f"[Prune] Pruned node at depth {node.depth}. New val acc: {best_acc:.4f}")
                break  # restart scanning nodes
            else:
                # revert
                if parent is None:
                    # revert root
                    root = TreeNode(is_leaf=backup[0], prediction=backup[1], feature=backup[2], threshold=backup[3], left=backup[4], right=backup[5], depth=node.depth)
                else:
                    setattr(parent, attr_name, node)  # restore original node
        # end for nodes
    print(f"[Prune] Final validation acc after pruning: {best_acc:.4f}")
    return root


In [6]:
def evaluate_model(root, X_tr, y_tr, X_v, y_v, X_te, y_te):
    ypred_tr = predict_tree(root, X_tr.values if isinstance(X_tr, pd.DataFrame) else X_tr, )
    ypred_v = predict_tree(root, X_v.values if isinstance(X_v, pd.DataFrame) else X_v)
    ypred_te = predict_tree(root, X_te.values if isinstance(X_te, pd.DataFrame) else X_te)
    print("Train Acc:", metrics.accuracy_score(y_tr, ypred_tr))
    print("Val   Acc:", metrics.accuracy_score(y_v, ypred_v))
    print("Test  Acc:", metrics.accuracy_score(y_te, ypred_te))
    print("\nTest classification report:")
    print(metrics.classification_report(y_te, ypred_te, digits=4))
    print("Confusion matrix:\n", metrics.confusion_matrix(y_te, ypred_te))
    return ypred_te

# wrapper to fit and optionally prune
def fit_and_eval(X_train_df, y_train_arr, X_val_df, y_val_arr, X_test_df, y_test_arr,
                 measure='gini', max_depth=None, min_samples_split=2, min_impurity_decrease=0.0, do_post_prune=False):
    X_train = X_train_df.values if isinstance(X_train_df, pd.DataFrame) else X_train_df
    X_val = X_val_df.values if isinstance(X_val_df, pd.DataFrame) else X_val_df
    X_test = X_test_df.values if isinstance(X_test_df, pd.DataFrame) else X_test_df

    t0 = time.time()
    root = build_tree(X_train, y_train_arr, measure=measure, max_depth=max_depth,
                      min_samples_split=min_samples_split, min_impurity_decrease=min_impurity_decrease)
    t_build = time.time() - t0
    print(f"Built tree (measure={measure}, max_depth={max_depth}, min_samples={min_samples_split}) in {t_build:.2f}s")
    # evaluation before prune
    print("Before pruning:")
    evaluate_model(root, X_train_df, y_train_arr, X_val_df, y_val_arr, X_test_df, y_test_arr)

    if do_post_prune:
        root = reduced_error_prune(root, X_val, y_val_arr)
        print("After pruning:")
        evaluate_model(root, X_train_df, y_train_arr, X_val_df, y_val_arr, X_test_df, y_test_arr)
    return root


In [7]:
configs = []
depths = [2, 4, 6, None]
measures = ['gini', 'entropy']
min_samples_split = 5  # pre-pruning requirement from your instructions

results = {}
for measure in measures:
    for d in depths:
        print("==========================================")
        print(f"Measure={measure}, max_depth={d}")
        root = fit_and_eval(X_train, y_train.values if isinstance(y_train, pd.Series) else y_train,
                            X_val, y_val.values if isinstance(y_val, pd.Series) else y_val,
                            X_test, y_test.values if isinstance(y_test, pd.Series) else y_test,
                            measure=measure, max_depth=d, min_samples_split=min_samples_split, do_post_prune=False)
        # store a lightweight record: validation accuracy
        ypred_val = predict_tree(root, X_val.values)
        val_acc = metrics.accuracy_score(y_val, ypred_val)
        results[(measure, d)] = {'root': root, 'val_acc': val_acc}


Measure=gini, max_depth=2
Built tree (measure=gini, max_depth=2, min_samples=5) in 24.17s
Before pruning:
Train Acc: 0.5482824708830901
Val   Acc: 0.5481481481481482
Test  Acc: 0.5472636815920398

Test classification report:
              precision    recall  f1-score   support

           0     0.5522    0.9572    0.7004      4531
           1     0.0000    0.0000    0.0000      2272
           2     0.5147    0.4081    0.4553      1502
           3     0.0000    0.0000    0.0000       740

    accuracy                         0.5473      9045
   macro avg     0.2667    0.3413    0.2889      9045
weighted avg     0.3621    0.5473    0.4264      9045

Confusion matrix:
 [[4337    0  194    0]
 [2162    0  110    0]
 [ 889    0  613    0]
 [ 466    0  274    0]]
Measure=gini, max_depth=4


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Built tree (measure=gini, max_depth=4, min_samples=5) in 36.64s
Before pruning:
Train Acc: 0.561624649859944
Val   Acc: 0.5600884466556109
Test  Acc: 0.560530679933665

Test classification report:
              precision    recall  f1-score   support

           0     0.5689    0.9473    0.7109      4531
           1     0.0000    0.0000    0.0000      2272
           2     0.5198    0.5146    0.5172      1502
           3     0.3571    0.0068    0.0133       740

    accuracy                         0.5605      9045
   macro avg     0.3615    0.3672    0.3103      9045
weighted avg     0.4005    0.5605    0.4431      9045

Confusion matrix:
 [[4292    0  238    1]
 [2145    0  127    0]
 [ 721    0  773    8]
 [ 386    0  349    5]]
Measure=gini, max_depth=6


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Built tree (measure=gini, max_depth=6, min_samples=5) in 47.61s
Before pruning:
Train Acc: 0.5695120153324488
Val   Acc: 0.5661691542288557
Test  Acc: 0.564400221116639

Test classification report:
              precision    recall  f1-score   support

           0     0.5761    0.9453    0.7159      4531
           1     0.1724    0.0022    0.0043      2272
           2     0.5196    0.5373    0.5283      1502
           3     0.3571    0.0135    0.0260       740

    accuracy                         0.5644      9045
   macro avg     0.4063    0.3746    0.3186      9045
weighted avg     0.4474    0.5644    0.4496      9045

Confusion matrix:
 [[4283   11  236    1]
 [2141    5  126    0]
 [ 667   11  807   17]
 [ 344    2  384   10]]
Measure=gini, max_depth=None
Built tree (measure=gini, max_depth=None, min_samples=5) in 95.83s
Before pruning:
Train Acc: 0.7383532360312546
Val   Acc: 0.4790491984521835
Test  Acc: 0.4705362078496407

Test classification report:
              precision 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Built tree (measure=entropy, max_depth=4, min_samples=5) in 51.15s
Before pruning:
Train Acc: 0.5612192245319181
Val   Acc: 0.5603095632946379
Test  Acc: 0.5609729132117192

Test classification report:
              precision    recall  f1-score   support

           0     0.5691    0.9473    0.7110      4531
           1     0.0000    0.0000    0.0000      2272
           2     0.5203    0.5206    0.5205      1502
           3     0.0000    0.0000    0.0000       740

    accuracy                         0.5610      9045
   macro avg     0.2723    0.3670    0.3079      9045
weighted avg     0.3715    0.5610    0.4426      9045

Confusion matrix:
 [[4292    0  239    0]
 [2145    0  127    0]
 [ 720    0  782    0]
 [ 385    0  355    0]]
Measure=entropy, max_depth=6


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Built tree (measure=entropy, max_depth=6, min_samples=5) in 50.89s
Before pruning:
Train Acc: 0.5689591626124134
Val   Acc: 0.5649530127142067
Test  Acc: 0.5637368711995577

Test classification report:
              precision    recall  f1-score   support

           0     0.5717    0.9525    0.7146      4531
           1     0.1724    0.0022    0.0043      2272
           2     0.5360    0.5153    0.5255      1502
           3     0.1739    0.0054    0.0105       740

    accuracy                         0.5637      9045
   macro avg     0.3635    0.3689    0.3137      9045
weighted avg     0.4329    0.5637    0.4472      9045

Confusion matrix:
 [[4316   11  201    3]
 [2164    5  102    1]
 [ 702   11  774   15]
 [ 367    2  367    4]]
Measure=entropy, max_depth=None
Built tree (measure=entropy, max_depth=None, min_samples=5) in 105.33s
Before pruning:
Train Acc: 0.7323087129588678
Val   Acc: 0.46943062465450525
Test  Acc: 0.4695411829740188

Test classification report:
            

In [10]:
# Build a deep/full tree (min_samples_split kept small to let it grow)
root_full = build_tree(X_train.values, y_train, measure='gini', max_depth=None, min_samples_split=2)
print("Validation acc before pruning:", metrics.accuracy_score(y_val, predict_tree(root_full, X_val.values)))
root_pruned = reduced_error_prune(root_full, X_val.values, y_val)
print("Validation acc after pruning:", metrics.accuracy_score(y_val, predict_tree(root_pruned, X_val.values)))
print("Final test results after pruning:")
evaluate_model(root_pruned, X_train, y_train, X_val, y_val, X_test, y_test)

KeyboardInterrupt: 

In [ ]:
def traverse_and_list_features(node, features, counts=None, depth_limit=3):
    if counts is None:
        counts = {}
    if node is None or node.is_leaf:
        return counts
    feat = features[node.feature] if node.feature is not None else None
    counts[feat] = counts.get(feat, 0) + 1
    traverse_and_list_features(node.left, features, counts, depth_limit-1)
    traverse_and_list_features(node.right, features, counts, depth_limit-1)
    return counts

# example using one trained tree (pick one from results)
sample_key = ('gini', 4)  # choose whatever you ran above
root_sample = results[sample_key]['root']
feature_list = X.columns.tolist()
feat_counts = traverse_and_list_features(root_sample, feature_list)
print("Top features used (counts):")
pprint.pprint(sorted(feat_counts.items(), key=lambda x: -x[1]))


In [11]:
# Fit sklearn with same depth options and report results
for measure in ['gini', 'entropy']:
    for d in [2,4,6,None]:
        clf = SklearnDT(criterion=measure, max_depth=d, min_samples_split=5, random_state=42)
        clf.fit(X_train, y_train)
        ypred_test = clf.predict(X_test)
        acc = metrics.accuracy_score(y_test, ypred_test)
        print(f"sklearn criterion={measure}, max_depth={d} -> Test Acc: {acc:.4f}")
    print("----")
# also output classification report for the best sklearn (unlimited)
clf = SklearnDT(criterion='gini', random_state=42)
clf.fit(X_train, y_train)
print("Sklearn full tree classification report on test:")
print(metrics.classification_report(y_test, clf.predict(X_test)))
print("Sklearn confusion matrix:\n", metrics.confusion_matrix(y_test, clf.predict(X_test)))


sklearn criterion=gini, max_depth=2 -> Test Acc: 0.5473
sklearn criterion=gini, max_depth=4 -> Test Acc: 0.5606
sklearn criterion=gini, max_depth=6 -> Test Acc: 0.5646
sklearn criterion=gini, max_depth=None -> Test Acc: 0.4612
----
sklearn criterion=entropy, max_depth=2 -> Test Acc: 0.5473
sklearn criterion=entropy, max_depth=4 -> Test Acc: 0.5610
sklearn criterion=entropy, max_depth=6 -> Test Acc: 0.5637
sklearn criterion=entropy, max_depth=None -> Test Acc: 0.4531
----
Sklearn full tree classification report on test:
              precision    recall  f1-score   support

           0       0.58      0.57      0.57      4531
           1       0.29      0.30      0.29      2272
           2       0.42      0.42      0.42      1502
           3       0.20      0.20      0.20       740

    accuracy                           0.45      9045
   macro avg       0.37      0.37      0.37      9045
weighted avg       0.45      0.45      0.45      9045

Sklearn confusion matrix:
 [[2569 1403  